In [1]:
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
tqdm.pandas()

import pandas as pd


In [2]:
ciudades = pd.read_html('https://es.wikipedia.org/wiki/Regi%C3%B3n_de_Los_Lagos')[3]
ciudades.drop(columns=['Bandera', 'Escudo'], inplace=True)
ciudades['Comunas'] = ciudades.apply(lambda row: row['Comunas'][2:], axis=1)
ciudades.index += 1 

## Mapa de la región de Los Lagos


<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/81/Los_Lagos_Comunas.svg/450px-Los_Lagos_Comunas.svg.png" width="100%" />

Fuente: [wikipedia](https://es.wikipedia.org/wiki/Portal:Regi%C3%B3n_de_Los_Lagos/Comunas_de_Los_Lagos)

In [3]:
ciudades

,Provincia,Capital,Comunas,Población
1,Chiloé,Castro,Ancud,38.991
2,Chiloé,Castro,Castro,33.417
3,Chiloé,Castro,Chonchi,5.632
4,Chiloé,Castro,Curaco de Vélez,3.829
5,Chiloé,Castro,Dalcahue,7.120
6,Chiloé,Castro,Puqueldón,3.921
7,Chiloé,Castro,Queilén,5.385
8,Chiloé,Castro,Quemchi,8.352
9,Chiloé,Castro,Quellón,27.192
10,Chiloé,Castro,Quinchao,8.088


## 1. Modelado de tópicos con BERTopic

In [4]:
from utils import filter_by_media
from utils import cluster_by_month
from utils import preprocess


df = pd.read_csv("data/loslagos-comunas.csv")[:100]
df = cluster_by_month(filter_by_media(df))
df['tokens'] =  df.content.progress_apply(lambda x: preprocess(str(x)))
df.isna().any()

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:06<00:00, 14.87it/s]


date               False
media_outlet       False
url                False
title              False
text               False
content            False
comuna              True
date_clustering    False
tokens             False
dtype: bool

## 2. Análisis de sentimiento 

In [ ]:
#!pip install pysentimiento

In [5]:
sub = df.copy()

In [10]:
from pysentimiento import create_analyzer
sentiment_analyzer = create_analyzer(task="sentiment", lang="es")
emotion_analyzer = create_analyzer(task="emotion", lang="es")
#hate_speech_analyzer = create_analyzer(task="hate_speech", lang="es")

sub['title_sentiment'] = ""
sub['title_emotion'] = ""
#sub['title_hate_speech'] = ""

for index, row in tqdm(sub.iterrows(), desc='sub rows - sentiment', total=sub.shape[0]):
    # Insertamos en dataframe
    sub.at[index, "title_sentiment"] = sentiment_analyzer.predict(row['title'])
    sub.at[index, "title_emotion"] = emotion_analyzer.predict(row['title'])
    #sub.at[index, "title_hate_speech"] = hate_speech_analyzer.predict(row['title'])


ConnectionError: ('Connection aborted.', TimeoutError(10060, 'Se produjo un error durante el intento de conexión ya que la parte conectada no respondió adecuadamente tras un periodo de tiempo, o bien se produjo un error en la conexión establecida ya que el host conectado no ha podido responder', None, 10060, None))

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None, 'display.max_colwidth', None)
sub[['title','title_sentiment', 'title_emotion', 'title_hate_speech']]
